In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import date
pd.set_option('display.max_columns', None)
import json

In [2]:
df_work_raw = catalog.load('raw/openalex/work_dev#parquet')

[12/05/24 15:00:13] INFO     Loading data from raw/openalex/work_dev#parquet                    ]8;id=759099;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=632386;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

# Profiling

In [3]:
df_work_raw.shape

(150, 50)

In [4]:
df_work_raw.head(3)

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'doi': 'https://doi.org/10.1016/s0378-4371(98...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': False, 'is_oa'...",[{'affiliations': [{'institution_ids': array([...,[],3,6,[https://openalex.org/A5051451815],[https://openalex.org/I4210125245],"{'currency': 'USD', 'provenance': 'doaj', 'val...",None,30.222,True,ngrams,1330,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '534', 'issue': '3-4', 'last_pa...",False,False,{'display_name': 'Statistical Mechanics with L...,[{'display_name': 'Statistical Mechanics with ...,[{'display_name': 'Hamiltonian (control theory...,"[{'display_name': 'Harmonic oscillator', 'id':...",[],1,"[{'is_accepted': False, 'is_oa': False, 'is_pu...",None,[{'display_name': 'Affordable and clean energy...,[],[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...",None,https://api.openalex.org/works?filter=cites:W2...,"[{'cited_by_count': 42, 'year': 2024}, {'cited...",2024-11-20T08:52:24.024876,2016-06-24
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'doi': 'https://doi.org/10.1016/j.envint.2020...,en,"{'is_accepted': True, 'is_oa': True, 'is_publi...",review,journal-article,"[crossref, doaj, pubmed]","{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],1,5,[https://openalex.org/A5069680875],"[https://openalex.org/I128590013, https://open...","{'currency': 'USD', 'provenance': 'doaj', 'val...","{'currency': 'USD', 'provenance': 'doaj', 'val...",8.533,True,pdf,506,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '105876', 'issue': None, 'last_...",False,False,{'display_name': 'Health Effects of Air Pollut...,[{'display_name': 'Health Effects of Air Pollu...,"[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'descriptor_name': 'Air Pollutants', 'descri...",4,"[{'is_accepted': True, 'is_oa': True, 'is_publ...","{'is_accepted': True, 'is_oa': True, 'is_publi...",[{'display_name': 'Good health and well-being'...,"[{'award_id': None, 'funder': 'https://openale...",[],[],84,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W4399338999, https://ope...","{'""Best': None, '""Good': None, '""Natural': Non...",https://api.openalex.org/works?filter=cites:W3...,"[{'cited_by_count': 154, 'year': 2024}, {'cite...",2024-12-03T13:35:34.799193,2020-07-02
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'doi': 'https://doi.org/10.1016/s0023-6438(03...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': False, 'is_oa'...",[{'a

In [5]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw.dtypes


id                                string[python]
doi                               string[python]
title                             string[python]
display_name                      string[python]
publication_year                           Int64
publication_date                  string[python]
ids                                       object
language                          string[python]
primary_location                          object
type                              string[python]
type_crossref                     string[python]
indexed_in                                object
open_access                               object
authorships                               object
institution_assertions                    object
countries_distinct_count                   Int64
institutions_distinct_count                Int64
corresponding_author_ids                  object
corresponding_institution_ids             object
apc_list                                  object
apc_paid           

# Nodo

In [ ]:
def land_work2authorships_openalex(df_work_raw):
    df_work2authorships = df_work_raw.loc[:,['id','authorships']]
    df_work2authorships = df_work2authorships.convert_dtypes()
    df_work2authorships.rename(columns={"id":'work_id'}, inplace=True)

    df_work2authorships_exploded = df_work2authorships.explode('authorships').reset_index(drop=True)
    df_work2authorships_norm =  pd.json_normalize(df_work2authorships_exploded['authorships'])
    df_work2authorships_norm.rename(columns={"author.id":'author_id'}, inplace=True)
    
    df_work = df_work2authorships_exploded.loc[:,'work_id']
    df_work2authorships = pd.concat((df_work, df_work2authorships_norm), axis=1)

    df_work2author = df_work2authorships.loc[:,['work_id','author_id','author_position']]
    df_author2institution_exploded = df_work2authorships.loc[:,['author_id','institutions']].explode('institutions').reset_index(drop=True)

    df_author = df_author2institution_exploded.loc[:,'author_id']
    df_institution_norm = pd.json_normalize(df_author2institution_exploded['institutions'])
    df_institution_norm.drop(columns=['lineage'], inplace=True)
    df_author2institution = pd.concat((df_author, df_institution_norm), axis=1)

    return df_work2author, df_author2institution


## Ejecuto Nodo

In [7]:
df_work2author, df_author2institution = land_work2authorships_openalex(df_work_raw)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df_work2author, df_author2institution = land_work2authorships_openalex(df_work_raw)          │
│   2                                                                                              │
│                                                                                                  │
│ in land_work2authorships_openalex:13                                                             │
│                                                                                                  │
│   10 │   df_work = df_work2authorships_exploded.loc[:,'work_id']                                 │
│   11 │   df_work2authorships = pd.concat((df_work, df_work2authorships_norm), axis=1)            │
│   12 │                                                                                           │
│ ❱ 13 │   df_work2author = df_work2authorships.loc[:,['work_id','author.id','author_position']    │
│   14 │   df_author2institution_exploded = df_work2authorships.loc[:,['author.id','institution    │
│   15 │                                                                                           │
│   16 │   df_author = df_author2institution_exploded.loc[:,'author.id']                           │
│                                                                                                  │
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/pandas/core/indexi │
│ ng.py:1184 in __getitem__                                                                        │
│                                                                                                  │
│   1181 │   │   │   key = tuple(com.apply_if_callable(x, self.obj) for x in key)                  │
│   1182 │   │   │   if self._is_scalar_access(key):                                               │
│   1183 │   │   │   │   return self.obj._get_value(*key, takeable=self._takeable)                 │
│ ❱ 1184 │   │   │   return self._getitem_tuple(key)                                               │
│   1185 │   │   else:                                                                             │
│   1186 │   │   │   # we by definition only have the 0th axis                                     │
│   1187 │   │   │   axis = self.axis or 0                                                         │
│                                                                                                  │
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/pandas/core/indexi │
│ ng.py:1377 in _getitem_tuple                                                                     │
│                                                                                                  │
│   1374 │   │   if self._multi_take_opportunity(tup):                                             │
│   1375 │   │   │   return self._multi_take(tup)                                                  │
│   1376 │   │                                                                                     │
│ ❱ 1377 │   │   return self._getitem_tuple_same_dim(tup)                                          │
│   1378 │                                                                                         │
│   1379 │   def _get_label(self, label, axis: AxisInt):                                           │
│   1380 │   │   # GH#5567 this will fail if the label is not present in the axis.                 │
│                                                                                                  │
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/pandas/core/indexi │
│ ng.py:1020 in _getitem_tuple_same_dim                      

# Resultados

In [ ]:
df_work2author

,work_id,author.id,author_position
0,https://openalex.org/W2089852843,https://openalex.org/A5051451815,first
1,https://openalex.org/W2089852843,https://openalex.org/A5110175901,middle
2,https://openalex.org/W2089852843,https://openalex.org/A5059654447,last
3,https://openalex.org/W3037375387,https://openalex.org/A5069680875,first
4,https://openalex.org/W3037375387,https://openalex.org/A5031160102,middle
...,...,...,...
934,https://openalex.org/W2892467442,https://openalex.org/A5082889193,middle
935,https://openalex.org/W2892467442,https://openalex.org/A5080242549,middle
936,https://openalex.org/W2892467442,https://openalex.org/A5088723412,middle
937,https://openalex.org/W2892467442,https://openalex.org/A5010003236,middle


In [ ]:
df_author2institution

,author.id,country_code,display_name,id,ror,type
0,https://openalex.org/A5051451815,BR,Centro Brasileiro de Pesquisas Físicas,https://openalex.org/I4210125245,https://ror.org/02wnmk332,facility
1,https://openalex.org/A5110175901,BR,Universidade Estadual de Maringá,https://openalex.org/I123443094,https://ror.org/04bqqa360,education
2,https://openalex.org/A5059654447,AR,Universidad Nacional de La Plata,https://openalex.org/I874386039,https://ror.org/01tjs6929,education
3,https://openalex.org/A5059654447,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government
4,https://openalex.org/A5059654447,AR,Consejo Nacional de Investigaciones Científica...,https://openalex.org/I151201029,https://ror.org/03cqe8w59,government
...,...,...,...,...,...,...
1610,https://openalex.org/A5010003236,AR,Universidad Nacional de La Plata,https://openalex.org/I874386039,https://ror.org/01tjs6929,education
1611,https://openalex.org/A5010003236,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government
1612,https://openalex.org/A5063484288,AR,Instituto Multidisciplinario de Biología Celular,https://openalex.org/I4210165869,https://ror.org/05p6vdy67,facility
1613,https://openalex.org/A5063484288,AR,Comisión de Investigaciones Científicas,https://openalex.org/I4210122970,https://ror.org/02s7sax82,government
